In [1]:
import pickle

In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl

import numpy as np
%matplotlib inline
import pandas as pd

In [3]:
import os, glob
import seaborn as sns
sns.set(style="darkgrid")

In [4]:
from IPython.display import display, HTML, Image

In [5]:
import plotly
import plotly.express as px
import plotly.graph_objects as go

In [6]:
#data5H =pd.read_csv('15_PCc_02_LHS_500_54854_01_s1_G.csv')
InsFile='10_PMU_02_LHS_500000_54854_02_t_P.dat'
data = pd.read_csv(InsFile,sep ='\t')

In [7]:
data.columns =[col.strip() for col in data.columns]
data.columns =[col.strip() for col in data.columns]
data =data.iloc[:-1,:-1]

## Label the dataset based


### Four gorups:

### Group1: acid with metaschoepite
### Group2: acid without metaschoepite
### Group3: base with metaschoepite
### Group4: base without metaschoepite

In [8]:
conds = [((data['metaschoepite']>0) & (data['totAcid']>1.0e-9)),
              ((data['metaschoepite']==0) & (data['totAcid']>1.0e-9)),
              ((data['metaschoepite']>0) & (data['totAcid']<=1.0e-9)),
              ((data['metaschoepite']==0) & (data['totAcid']<=1.0e-9)),
]
groups = ['group1','group2','group3','group4']

data['group'] =np.select(conds, groups)


## Supervised grouping with ML models

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [10]:
# metrics are used to find accuracy or error
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler

In [11]:
data.columns

Index(['mass_H2O', 'totU', 'totAcid', 'totBase', 'pH', 'U_aq', 'U_s', 'U_sc',
       'U_ex', 'Kd_s', 'Kd_sc', 'Kd_ex', 'metaschoepite', 'group'],
      dtype='object')

In [12]:
inputVars= ['totU', 'totAcid', 'totBase']
targVars = ['group']

In [13]:
X = data[inputVars].values
y = data[targVars].values

In [14]:
scaler = StandardScaler().fit(X)
X_scaled =  scaler.transform(X)

In [15]:
pickle.dump(scaler, open('scaler_grouping.pkl', 'wb'))
print(scaler.__dict__)


{'with_mean': True, 'with_std': True, 'copy': True, 'n_features_in_': 3, 'n_samples_seen_': 499999, 'mean_': array([0.00062041, 0.00499999, 0.00500001]), 'var_': array([2.71716870e-06, 4.16666374e-05, 4.16667205e-05]), 'scale_': array([0.00164838, 0.00645497, 0.00645498])}


In [16]:
# i.e. 80 % training dataset and 20 % test datasets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.20)

In [17]:
# creating a RF classifier
clf = RandomForestClassifier(n_estimators = 2000) 
# Training the model on the training dataset
# fit function is used to train the model using the training sets as parameters
clf.fit(X_train, y_train.ravel())
 
# performing predictions on the test dataset
y_pred = clf.predict(X_test)
 

print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))

ACCURACY OF THE MODEL:  0.9997


In [18]:
pickle.dump(clf, open('randomforest_grouping.pkl', 'wb'))

In [19]:
## Test with 5K dataset

In [20]:
#data5H =pd.read_csv('15_PCc_02_LHS_500_54854_01_s1_G.csv')
InsFile='10_PMU_02_LHS_5000_54854_02_t_P.dat'
data5K = pd.read_csv(InsFile,sep ='\t')
data5K.columns =[col.strip() for col in data5K.columns]
data5K.columns =[col.strip() for col in data5K.columns]
data5K =data5K.iloc[:-1,:-1]

conds = [((data5K['metaschoepite']>0) & (data5K['totAcid']>1.0e-9)),
              ((data5K['metaschoepite']==0) & (data5K['totAcid']>1.0e-9)),
              ((data5K['metaschoepite']>0) & (data5K['totAcid']<=1.0e-9)),
              ((data5K['metaschoepite']==0) & (data5K['totAcid']<=1.0e-9)),
]
groups = ['group1','group2','group3','group4']

data5K['group'] =np.select(conds, groups)


In [21]:
X = data5K[inputVars].values
y = data5K[targVars].values
X_scaled =  scaler.transform(X)
y_pred_5K = clf.predict(X_scaled)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y, y_pred_5K))

ACCURACY OF THE MODEL:  0.9995999199839968
